# Import data& libraries 

In [200]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

In [201]:
path= "/mnt/d/Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
X_train = X_train.sample(frac=1)
y = X_train["N_category"]
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

In [202]:
X_train.head(5)

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
947,42,2010-08-04,1,1,1,15.0,2.0,2.0,2.0,2.0,...,1.0,5.0,1.0,4.0,NaN,1.0,2.0,NaN,NaN,NaN
666,51,2011-10-17,1,1,1,11.0,3.0,2.0,3.0,3.0,...,0.0,NaN,0.0,NaN,10.00,0.0,1.0,NaN,NaN,NaN
259,55,2019-05-09,1,2,1,33.0,3.0,3.0,3.0,3.0,...,1.0,8.0,1.0,8.0,26.34,0.0,1.0,NaN,NaN,NaN
84,51,2019-12-05,1,2,2,21.0,2.0,2.0,3.0,2.0,...,1.0,7.0,0.0,NaN,18.86,0.0,2.0,0.0,1.27,NaN
743,40,2011-02-11,1,1,1,13.0,3.0,3.0,3.0,3.0,...,1.0,5.0,0.0,NaN,NaN,0.0,1.0,NaN,NaN,NaN


# Fill missing values

In [203]:
X_train = X_train.fillna(X_train.mean())

/tmp/ipykernel_369/3112579809.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


# Process categorical values

In [204]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

In [205]:
X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name()
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))
X_train["수술연월일"] = date_list

encoder = LabelEncoder()
X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

index = [i for i in range(28)if i != 24]
X_train = X_train[X_train.columns[index]]

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [206]:
original_X = X_train.copy()

In [207]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [208]:
from sklearn.ensemble import RandomForestClassifier

In [271]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [274]:
pred = model.predict(X_valid)
f1_score(pred, y_valid)

0.7804878048780487

# Data preprocessing 

In [249]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [298]:
def process_score(X, model):
    f1_list=[]
#     X_train, X_valid, y_train, y_valid = train_test_split(data, y, stratify=y, test_size=0.2)
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    skf.get_n_splits(X, y)    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model_ = RandomForestClassifier()
        model_.fit(X_train, y_train)
        pred = model_.predict(X_test)
        print(model_.score(X_test, y_test))
        f1_list.append(f1_score(y_test, pred, average="macro"))
    print(np.array(f1_list).mean())
    print(f1_list)

In [299]:
model = RandomForestClassifier()
process_score(original_X, model)

0.695
0.61
0.705
0.68
0.655
0.6679713874380153
[0.6915998887737304, 0.60984393757503, 0.7044014128610436, 0.6797117405665098, 0.6542999574137629]


# Test models

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

## Random forest

In [22]:
parameters = {'n_estimators':range(100 ,1000, 100),
              'max_depth':range(10, 100, 10),           
             }

In [25]:
model = RandomForestClassifier()
clf = GridSearchCV(model, parameters, n_jobs=-1)
clf.fit(X_train, y_train)


KeyboardInterrupt



In [217]:
clf.best_params_

{'max_depth': 10, 'n_estimators': 200}

In [220]:
process_score(original_X, clf.best_estimator_)

0.5794604497413361


/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic regression

In [203]:
parameters = {'penalty':["l1", "l2" "none", "elastiacnet"],
              'tol':[1e-3,5e-3, 1e-4, 5e-3] , 
              "C": [0.1, 0.3, 0.5, 1, 1.5],
              "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
              "max_iter": range(100, 1000, 100)
             }

In [205]:
model = LogisticRegression()
clf = GridSearchCV(model, parameters, n_jobs=-1)
clf.fit(X_train, y_train)

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaco

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaco

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaco

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaco

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl

...

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/e

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/e

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaco

/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/hykhhijk/anaconda3/envs/dl

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 0.3, 0.5, 1, 1.5],
                         'max_iter': range(100, 1000, 100),
                         'penalty': ['l1', 'l2none', 'elastiacnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga'],
                         'tol': [0.001, 0.005, 0.0001, 0.005]})

In [206]:
clf.best_params_

{'C': 0.5,
 'max_iter': 100,
 'penalty': 'l1',
 'solver': 'liblinear',
 'tol': 0.005}

In [212]:
process_score(original_X, clf.best_estimator_)

0.6986480890475327


## SVC

In [224]:
model_1 = LogisticRegression(C= 0.5,
 max_iter= 100,
 penalty= 'l1',
 solver= 'liblinear',
 tol= 0.005)

In [225]:
model_2 = RandomForestClassifier(max_depth=10, n_estimators=200)

In [228]:
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200)

In [235]:
pred_1 = model_1.predict(X_valid)
pred_2 = model_2.predict(X_valid)
proba_1 = model_1.predict_proba(X_valid)
proba_2= model_2.predict_proba(X_valid)


In [241]:
precision_recall_fscore_support(pred_1, y_valid)

(array([0.73195876, 0.7184466 ]),
 array([0.71, 0.74]),
 array([0.72081218, 0.72906404]),
 array([100, 100]))

In [243]:
precision_recall_fscore_support(pred_2, y_valid)

(array([0.84536082, 0.80582524]),
 array([0.80392157, 0.84693878]),
 array([0.8241206 , 0.82587065]),
 array([102,  98]))

In [250]:
last

array([[0.50150287, 0.49849713],
       [0.09442281, 0.90557719],
       [0.02606077, 0.97393923],
       [0.81644213, 0.18355787],
       [0.94130187, 0.05869813],
       [0.66738415, 0.33261585],
       [0.4379822 , 0.5620178 ],
       [0.0154054 , 0.9845946 ],
       [0.93159857, 0.06840143],
       [0.34773218, 0.65226782],
       [0.7033526 , 0.2966474 ],
       [0.69287097, 0.30712903],
       [0.32816526, 0.67183474],
       [0.83731302, 0.16268698],
       [0.72455384, 0.27544616],
       [0.07829956, 0.92170044],
       [0.93771684, 0.06228316],
       [0.2940917 , 0.7059083 ],
       [0.83585952, 0.16414048],
       [0.04593087, 0.95406913],
       [0.58577183, 0.41422817],
       [0.72499601, 0.27500399],
       [0.5040541 , 0.4959459 ],
       [0.68097239, 0.31902761],
       [0.30459521, 0.69540479],
       [0.08582128, 0.91417872],
       [0.1761563 , 0.8238437 ],
       [0.18996265, 0.81003735],
       [0.64519614, 0.35480386],
       [0.74569372, 0.25430628],
       [0.

In [258]:
temp=[]
for i in last:
    if i[0] <= 0.5:
        temp.append(1)
    else:
        temp.append(0)

In [259]:
temp

[0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1]

In [260]:
precision_recall_fscore_support(temp, y_valid)

(array([0.75257732, 0.75728155]),
 array([0.74489796, 0.76470588]),
 array([0.74871795, 0.76097561]),
 array([ 98, 102]))

In [262]:
X_test

,ID,img_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,BC_01_0011,./test_imgs/BC_01_0011.png,55,2015-11-17,2,2,1,23.0,2.0,2.0,...,1.0,8.0,0.0,2.0,5.00,0.0,1.0,NaN,NaN,NaN
1,BC_01_0220,./test_imgs/BC_01_0220.png,43,2020-06-09,4,2,1,13.0,3.0,2.0,...,1.0,4.0,1.0,8.0,8.67,0.0,0.0,NaN,NaN,NaN
2,BC_01_0233,./test_imgs/BC_01_0233.png,76,2020-05-14,1,1,1,NaN,NaN,NaN,...,1.0,6.0,1.0,6.0,NaN,NaN,2.0,NaN,NaN,NaN
3,BC_01_0258,./test_imgs/BC_01_0258.png,58,2020-05-20,1,2,1,1.3,2.0,2.0,...,1.0,7.0,0.0,NaN,21.17,1.0,3.0,NaN,NaN,0.0
4,BC_01_0260,./test_imgs/BC_01_0260.png,56,2020-05-20,1,2,2,15.0,3.0,3.0,...,1.0,8.0,1.0,3.0,20.57,1.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,BC_01_3328,./test_imgs/BC_01_3328.png,61,2009-10-30,1,1,1,18.0,2.0,2.0,...,1.0,5.0,1.0,3.0,NaN,0.0,0.0,NaN,NaN,NaN
246,BC_01_3404,./test_imgs/BC_01_3404.png,42,2009-05-19,1,2,1,20.0,1.0,2.0,...,1.0,5.0,1.0,5.0,NaN,0.0,1.0,NaN,NaN,NaN
247,BC_01_3418,./test_imgs/BC_01_3418.png,37,2009-04-24,1,1,1,17.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN
248,BC_01_3438,./test_imgs/BC_01_3438.png,37,2009-02-06,1,1,1,7.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN


In [272]:
def preprocess(X_train):
    X_train = X_train[X_train.columns[2:]]
    X_train = X_train.fillna(X_train.mean())    
    X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])
    X_train["year"] = X_train["수술연월일"].dt.year
    X_train["month"] = X_train["수술연월일"].dt.month
    X_train["day"] = X_train["수술연월일"].dt.day
    X_train["day_name"] = X_train["수술연월일"].dt.day_name()

    date_list=[]
    for i in range(len(X_train)):
        date_list.append(str(X_train["year"][i]).zfill(2)+str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

    X_train["수술연월일"] = date_list

    encoder = LabelEncoder()
    X_train["day_name"] = pd.DataFrame(encoder.fit_transform(np.reshape(np.array(X_train["day_name"]), (-1,1))))

    index = [i for i in range(28)if i != 24]
    X_train = X_train[X_train.columns[index]]
    return X_train

In [273]:
X_test = preprocess(X_test)

/tmp/ipykernel_445/4079215291.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())
/home/hykhhijk/anaconda3/envs/dl/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [286]:
pred1 = model_1.predict_proba(X_test)
pred2 = model_2.predict(X_test)

In [280]:
temp=[]
for i in last:
    if i[0] <= 0.5:
        temp.append(1)
    else:
        temp.append(0)

In [279]:
last = (pred1+pred2)/2

In [287]:
submission["N_category"] = pred2

In [288]:
submission.to_csv("./submission_tabular.csv", index=False)